In [1]:
import gymnasium as gym
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random
import math
import matplotlib.pyplot as plt
import numpy as np
from collections import deque, namedtuple
import cv2 as cv

c:\Python310\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


WHY THE HECK DOESNT THIS MATCH THE ORIGINAL CODE PERFORMANCE, I MUST HAVE A BUG SOMEWHERE NOOOOOOOOOOOOOOOOOOOOOOOOOOOO

In [2]:
environment     = gym.make("CartPole-v1", render_mode=None)
experience      = namedtuple("Experience", "state action nextState reward")
device          = torch.device("cuda" if torch.cuda.is_available() else "cpu")
observationSize = environment.observation_space.shape[0]
numActions      = environment.action_space.n

In [3]:
class LunarLander(nn.Module):
    def __init__(self, numObs, numAct):
        super().__init__()
        self.layer1 = nn.Linear(numObs, 128)
        self.layer2 = nn.Linear(128, 128)
        self.layer3 = nn.Linear(128, numAct)

    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        return self.layer3(x)

In [4]:
class Memory(object):
    def __init__(self, capacity):
        self.memory = deque(maxlen=capacity)

    def push(self, experience):
        self.memory.append(experience)

    def sample(self, batchSize):
        return random.sample(self.memory, batchSize)

    def __len__(self):
        return len(self.memory)

In [5]:
def epsilonGreedyPolicy(epsilonStart, epsilondEnd, epsilonDecay, episode, maxEpisodes):
    sample = random.random()
    threshold = epsilondEnd + (epsilonStart - epsilondEnd) * np.exp(-epsilonDecay*episode/maxEpisodes)
    if sample > threshold:
        return False
    return True

In [6]:
def softUpdate(policyModel, targetModel, softUpdateRate):
    targetModelStateDict = targetModel.state_dict()
    policyModelStateDict = policyModel.state_dict()
    for key in policyModelStateDict:
        targetModelStateDict[key] = policyModelStateDict[key]*softUpdateRate + targetModelStateDict[key]*(1 - softUpdateRate)
    targetModel.load_state_dict(targetModelStateDict)
    return targetModel

In [7]:
def policy(model, state):
    return torch.argmax(model(state)).item()

In [8]:
def saveVideo(images, outputDirectory, fps):
    height, width, _ = images[0].shape
    writer = cv.VideoWriter(outputDirectory, cv.VideoWriter_fourcc(*'H264'), fps, (width, height))
    for image in images:
        bgr_image = cv.cvtColor(image, cv.COLOR_RGB2BGR)
        writer.write(bgr_image)
    writer.release()

In [9]:
def optimizeModel(policyNet, targetNet, optimizer, memory, batchSize, discountFactor):
        experiences = memory.sample(min(batchSize, len(memory)))
        batch = experience(*zip(*experiences))
        stateBatch = torch.tensor(batch.state).to(device)
        actionBatch = torch.tensor(batch.action).to(device)
        rewardBatch = torch.tensor(batch.reward).to(device)
        Qexpected = policyNet(stateBatch).gather(1, actionBatch.view(-1, 1))
        QtargetsNext = torch.zeros(batchSize, device=device)
        with torch.no_grad():
            QtargetsNext = targetNet(stateBatch).max(1).values
        Qtargets = rewardBatch.float() + (QtargetsNext*discountFactor)
        criterion = nn.SmoothL1Loss()
        loss = criterion(Qexpected, Qtargets.unsqueeze(1))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [10]:
policyNet = LunarLander(observationSize, numActions).to(device)
targetNet = LunarLander(observationSize, numActions).to(device)
targetNet.load_state_dict(policyNet.state_dict())

<All keys matched successfully>

In [11]:
BATCH_SIZE          = 128
GAMMA               = 0.99
EPSILON_START       = 0.9
EPSILON_END         = 0.05
EPSILON_DECAY       = 6
TAU                 = 0.005
LEARNING_RATE       = 0.0001

optimizer = optim.AdamW(policyNet.parameters(), lr=LEARNING_RATE, amsgrad=True)
memory = Memory(10000)

In [12]:
nEpisodes = 500
episodeRewards = []
for i in range(nEpisodes):
    state, _ = environment.reset()
    totalReward = 0
    while True:
        if epsilonGreedyPolicy(EPSILON_START, EPSILON_END, EPSILON_DECAY, episode=i, maxEpisodes=nEpisodes) == True:
            action = environment.action_space.sample()
        else:
            action = policy(policyNet, torch.tensor(state, device=device))

        nextState, reward, terminated, truncated, _ = environment.step(action)
        memory.push(experience(state, action, nextState, reward))
        totalReward += reward

        if terminated or truncated:
            print(f"Episode: {i + 1:>5}, Final reward: {totalReward:>8.2f}")
            episodeRewards.append(totalReward)
            break

        state = nextState
        optimizeModel(policyNet, targetNet, optimizer, memory, BATCH_SIZE, GAMMA)
        #targetNet = softUpdate(policyNet, targetNet, TAU)
        targetModelStateDict = targetNet.state_dict()
        policyModelStateDict = policyNet.state_dict()
        for key in policyModelStateDict:
            targetModelStateDict[key] = policyModelStateDict[key]*TAU + targetModelStateDict[key]*(1 - TAU)
        targetNet.load_state_dict(targetModelStateDict)


C:\Users\Inexperienced Me\AppData\Local\Temp\ipykernel_71628\3836152365.py:4: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:281.)
  stateBatch = torch.tensor(batch.state).to(device)


Episode:     1, Final reward:    27.00
Episode:     2, Final reward:    13.00
Episode:     3, Final reward:    35.00
Episode:     4, Final reward:    12.00
Episode:     5, Final reward:    23.00
Episode:     6, Final reward:    21.00
Episode:     7, Final reward:    37.00
Episode:     8, Final reward:    34.00
Episode:     9, Final reward:    14.00
Episode:    10, Final reward:    13.00
Episode:    11, Final reward:    11.00
Episode:    12, Final reward:    23.00
Episode:    13, Final reward:    21.00
Episode:    14, Final reward:    17.00
Episode:    15, Final reward:    33.00
Episode:    16, Final reward:    12.00
Episode:    17, Final reward:    23.00
Episode:    18, Final reward:    24.00
Episode:    19, Final reward:    13.00
Episode:    20, Final reward:    16.00
Episode:    21, Final reward:    13.00
Episode:    22, Final reward:    17.00
Episode:    23, Final reward:    13.00
Episode:    24, Final reward:    10.00
Episode:    25, Final reward:    15.00
Episode:    26, Final rew

In [16]:
environment = gym.make("CartPole-v1", render_mode="rgb_array")
with torch.no_grad():
    images = []
    for i in range(1):
        state, _ = environment.reset()
        totalReward = 0
        while True:
            action = policy(policyNet, torch.tensor(state, device=device))
            nextState, reward, terminated, truncated, _ = environment.step(action)
            totalReward += reward
            images.append(environment.render())

            if terminated or truncated:
                print(f"Episode: {i + 1:>5}, Final reward: {totalReward:>8.2f}")
                break

            state = nextState
    saveVideo(images, "shouldBEGOOD.mp4", 30)

c:\Python310\lib\site-packages\pygame\pkgdata.py:25: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import resource_stream, resource_exists
c:\Python310\lib\site-packages\pkg_resources\__init__.py:2871: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('mpl_toolkits')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
c:\Python310\lib\site-packages\pkg_resources\__init__.py:2871: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(

Episode:     1, Final reward:    10.00
